# This notebook provides portfolio weights based on the historical data to the backtesting notebook

In [ ]:
!pip uninstall deepdowmine -y
!pip install git+https://github.com/dsman1823/deepdowmine.git

In [4]:
import torch

import pandas as pd
import numpy as np

from datetime import datetime 
from deepdowmine.nn import LinearNetMine, UpdNumericalMarkowitzWithShorting, UpdLinearNetMine, UpdDenseNet, UpdUpdLinearNetMine
from deepdowmine.nn import DenseNetMinVar, ConvNetFullOpti, ConvNetMinVar, RnnNetMinVar, LstmNetMinVar, RnnNetFullOpti
from deepdowmine.nn import LstmNetFullOpti, DenseNetFullOpti2, DenseNetMinVar2, RnnNetMinVar3, RnnNetFullOpti3, ConvNetMinVar2, ConvNetFullOpti2
from dateutil.relativedelta import relativedelta
from datetime import datetime, date

In [5]:
RETS_FILE_PATH = 'historical_returns.csv'



loockback, gap, horizon = 50, 0, 5
n_assets = 5
loockback = 50


## Load NN from dict

In [6]:
LINEAR_NET_OSHARPE_FILE_PATH = r'./NNs/linear_net_50x5_sharpe.pth' # LinearNetMine with ordinary Sharpe cost

In [7]:
# years parameter reprsenets amount of years after the backtest stasrt: the model dowloaded is the the model trained 
# on the period (train_start, back_test_start + years) 


# LinearNetMine with ordinary sharpe cost function
def load_linear_net_osharpe(years=0):
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{years}.pth'))
    print(fr'./NNs/linear_net_50x5_sharpe_{years}.pth')
    return network.eval()
    

In [8]:
def load_dense_net(years=0):
    network = UpdLinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/dense_{years}.pth'))
    print(fr'dense_{years}.pth')
    return network.eval()

## Core functions

In [9]:
def transform_rets_to_NN_input(rets):
    # tranform (loockback, n_assets) df into (1, 1, loockback, n_assets)=(sample_size, n_channels, loockback, n_assets) tensor 
    returns_np = rets.to_numpy()

    # Add the required dimensions: (n_samples, n_channels, lookback, n_assets)
    returns_np_expanded = np.expand_dims(returns_np, axis=0)  # Adds n_samples dimension
    returns_np_expanded = np.expand_dims(returns_np_expanded, axis=0)  
    return torch.from_numpy(returns_np_expanded).float()

def weights_from_NN(rets, network):
    X = transform_rets_to_NN_input(rets)
    return network(X).detach().numpy()[0]

# method is a function, used for obtaining the returns
def get_weights(method):
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    return method(rets)

## NNs and related methods


### NNs

networks[i] method is trained on (train_start, backtest_start + i years) data interval

In [10]:
years = [0, 1, 2]#, 1, 2]

linear_osharpe_networks = [load_linear_net_osharpe(years=i) for i in years]
dense_networks = [load_dense_net(years=i) for i in years]

FileNotFoundError: [Errno 2] No such file or directory: './NNs/linear_net_50x5_sharpe_0.pth'

### methods

In [ ]:
dense_networks[1].linear0.bias[0:3]

In [ ]:
dense_networks[2].linear0.bias[0:3]

In [ ]:
# note the format of lambda 
linear_osharpe_methods = [lambda r, i=i: weights_from_NN(r, linear_osharpe_networks[i]) for i in years] 
dense_methods = [lambda r, i=i: weights_from_NN(r, dense_networks[i]) for i in years] 


In [ ]:
[get_weights(dense_methods[i]) for i in years]

# Server

In [13]:


RETRAINING_DATE1 = date(2018, 12, 15)
RETRAINING_DATE2 = date(2019, 12, 15)

# RETRAINING_DATE1 = date(2021, 12, 15)
# RETRAINING_DATE2 = date(2022, 12, 15)

In [14]:
from flask import Flask, request, jsonify
from flask_cors import CORS 

import numpy as np

app = Flask(__name__)
CORS(app)  # Enable CORS for your Flask app

markers = {}
markers['net_initialized'] = False

@app.route('/get_weights', methods=['GET'])
def get_request():
    # Extract date string from the query parameters
    date_str = request.args.get('date', None)
    
    # Simple validation to check if date is provided
    if not date_str:
        return jsonify({"error": "Missing date parameter"}), 400

    # Try to convert the date string to a datetime object
    try:
        # Note that now we're only parsing the date, not the time
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        # If there is an error in parsing the date, return an error message
        return jsonify({"error": "Invalid date format. Please use YYYY-MM-DD format."}), 400
    
    years_plus = 0
    network = RnnNetMinVar3(5,  .15, None, 1)
    network.load_state_dict(torch.load(fr'./NNs/trained_on_old_data/rnn/min_var_0.pth'))

    
    if date < RETRAINING_DATE1:
        print(0)
    if RETRAINING_DATE1 < date < RETRAINING_DATE2:
        print('1')
        network.load_state_dict(torch.load(fr'./NNs/trained_on_old_data/rnn/min_var_1.pth'))
    if RETRAINING_DATE2 < date:
        print('2')
        network.load_state_dict(torch.load(fr'./NNs/trained_on_old_data/rnn/min_var_2.pth'))

    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    X = transform_rets_to_NN_input(rets)
    weights = network.eval()(X).detach().numpy()[0].tolist()
    
        
    #weights = weights_from_NN(rets, linear_osharpe_networks[0]).tolist()
    #print(weights)
    # Return the vector as JSON, using the string representation of the date for simplicity
    print(date)
    print(weights)
    return jsonify({'weights': weights})


In [ ]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2024 20:39:29] "GET /get_weights?date=2017-12-18 HTTP/1.1" 200 -


0
2017-12-18
[-0.07895562052726746, 0.610747754573822, -0.42514437437057495, 0.352143257856369, 0.5412089824676514]


127.0.0.1 - - [13/Apr/2024 20:39:32] "GET /get_weights?date=2017-12-26 HTTP/1.1" 200 -


0
2017-12-26
[0.1999300867319107, 0.029027244076132774, -0.446811318397522, 0.6569528579711914, 0.5609009265899658]


...

2
2020-11-30
[0.2592531740665436, 0.006374971941113472, 0.6209328174591064, 0.19418121874332428, -0.08074221760034561]


127.0.0.1 - - [13/Apr/2024 20:51:24] "GET /get_weights?date=2020-12-07 HTTP/1.1" 200 -


2
2020-12-07
[0.12571674585342407, 0.732507586479187, 0.06964726001024246, 0.29553449153900146, -0.22340624034404755]


127.0.0.1 - - [13/Apr/2024 20:51:26] "GET /get_weights?date=2020-12-14 HTTP/1.1" 200 -


2
2020-12-14
[0.007450304925441742, -0.18072398006916046, 0.20070527493953705, 0.9591578245162964, 0.01341050025075674]


In [ ]:
update the 